In [ ]:
import os, random
import numpy as np
import scanpy as sc
import pandas as pd
import anndata
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import tensorflow as tf


In [ ]:
from tensorflow.python.framework.ops import enable_eager_execution, disable_eager_execution

In [ ]:
disable_eager_execution()

# Load Data

In [ ]:
noise_file = "sim_g2_dropout_5"

obs_label_column = "Group"

adata = sc.read_h5ad(f"C:\\Users\\gmaho\\Desktop\\ai_masters_thesis\\simulated_datasets\\{noise_file}.h5ad")

In [ ]:
sim_raw = adata

y = np.array(adata.obs[obs_label_column])

sim_true = sc.AnnData(adata.layers["TrueCounts"],obs=pd.DataFrame(y, columns=[obs_label_column]))

In [ ]:
sc.pp.filter_genes(sim_raw, min_counts=1)

sim_true

sim_raw_norm = sim_raw.copy()
sc.pp.normalize_total(sim_raw_norm)
sc.pp.log1p(sim_raw_norm)
sc.pp.pca(sim_raw_norm)

sim_true_norm = sim_true.copy()
sc.pp.normalize_total(sim_true_norm)
sc.pp.log1p(sim_true_norm)
sc.pp.pca(sim_true_norm)

# print(dropout_gt[:10, :10])
print(sim_raw)
print(sim_true)

# Denoise Simulated Data

In [ ]:
def run_prep(adata,
        mode='denoise',
        hidden_size=(64, 32, 64), 
        hidden_dropout=0.,
        batchnorm=True,
        activation='relu',
        init='glorot_uniform',
        epochs=300, 
        reduce_lr=10,
        early_stop=20,
        batch_size=32,
        optimizer='RMSprop',
        learning_rate=None,
        random_state=0,
        threads=None,
        verbose=False,
        ):
    

    assert isinstance(adata, anndata.AnnData), 'adata must be an AnnData instance'
    assert mode in ('denoise', 'latent'), '%s is not a valid mode.' % mode

    # set seed for reproducibility
    random.seed(random_state)
    np.random.seed(random_state)
    tf.random.set_seed(random_state)

    # check for zero genes
    nonzero_genes, _ = sc.pp.filter_genes(adata.X, min_counts=1)
    assert nonzero_genes.all(), 'Please remove all-zero genes before using DCA.'

    adata.raw = adata.copy()

    sc.pp.normalize_total(adata, key_added="n_counts")
    adata.obs["sf"] = adata.obs.n_counts / np.median(adata.obs.n_counts)

    sc.pp.scale(adata)

    input_size = output_size = adata.n_vars

    network_keywords = {
                    'input_size': input_size,
                    'output_size': output_size,
                    'hidden_size': hidden_size,
                    'hidden_dropout': hidden_dropout,
                    'batchnorm': batchnorm,
                    'activation': activation,
                    'init': init
                    }

    

    # ------------------------------------------------
    training_keywords = {
                     'epochs': epochs,
                     'reduce_lr': reduce_lr,
                     'early_stop': early_stop,
                     'batch_size': batch_size,
                     'optimizer': optimizer,
                     'verbose': verbose,
                     'threads': threads,
                     'learning_rate': learning_rate
                     }
    #-----------------------------------------------------------------------
    # adata.obs['split'] = 'train'

    # training_data = adata[adata.obs.split == 'train']

    return adata, network_keywords, training_keywords

In [ ]:
adata, network_keywords, training_keywords = run_prep(sim_raw)

In [ ]:
network_keywords

# Building a basic AE model

## Imports

In [ ]:
import numpy as np
import scanpy as sc

import keras
from keras.layers import Input, Dense, Activation, BatchNormalization, Lambda, Layer
from keras.models import Model
from keras.losses import MeanSquaredError

import tensorflow as tf

In [ ]:
def _nan_to_infinite(x):
    return tf.where(tf.math.is_nan(x), tf.zeros_like(x)+np.inf, x)

Multiply = Lambda(lambda l: l[0]*tf.reshape(l[1], (-1,1)))
MeanActivation = lambda x: tf.clip_by_value(tf.math.exp(x), 1e-5, 1e6)
VarActivation = lambda x: tf.clip_by_value(tf.nn.softplus(x), 1e-4, 1e4)

class SliceLayer(Layer):
    def __init__(self, index, **kwargs):
        self.index = index
        super().__init__(**kwargs)

    def build(self, input_shape):
        if not isinstance(input_shape, list):
            raise ValueError('Input should be a list')

        super().build(input_shape)

    def call(self, x):
        assert isinstance(x, list), 'SliceLayer input is not a list'
        return x[self.index]

    def compute_output_shape(self, input_shape):
        return input_shape[self.index]

# Negative Binomial Loss Function attempt
class NbLoss(object):
    def __init__(self, theta=None, scope="nb-loss/",
                 scale=1.0):
        
        # epsilon present for stability
        self.epsilon = 1e-10
        self.scale = scale
        self.theta = theta
        self.scope = scope

    def loss(self, y_true, y_pred, mean=True):
        scale = self.scale
        epsilon = self.epsilon

        with tf.name_scope(self.scope):
            y_true = tf.cast(y_true, tf.float32)
            y_pred = tf.cast(y_pred, tf.float32) * scale

            theta = tf.minimum(self.theta, 1e6)

            term1 = tf.math.lgamma(theta+epsilon) + tf.math.lgamma(y_true+1.0) - tf.math.lgamma(y_true+theta+epsilon)
            term2 = (theta+y_true) * tf.math.log(1.0 + (y_pred/(theta+epsilon))) + (y_true * (tf.math.log(theta+epsilon) - tf.math.log(y_pred+epsilon)))

            final = term1 + term2

            final = _nan_to_infinite(final)

            if mean:
                final = tf.reduce_mean(final)


        return final

# Zero inflated Negative Binomial Loss Function
class ZiNbLoss(NbLoss):
    def __init__(self, pmw, lambda_=0.0, scope="zinb-loss", **kwargs):
        super().__init__(scope=scope, **kwargs)
        self.pmw = pmw
        self.lambda_ = lambda_

    def loss(self, y_true, y_pred, mean=True):
        scale = self.scale
        epsilon = self.epsilon

        with tf.name_scope(self.scope):
            nb_case = super().loss(y_true, y_pred, mean=False) - tf.math.log(1.0-self.pmw+epsilon)

            y_true = tf.cast(y_true, tf.float32)
            y_pred = tf.cast(y_pred, tf.float32) * scale
            theta = tf.minimum(self.theta, 1e6)

            zero_nb = tf.pow(theta/(theta+y_pred+epsilon), theta)
            zero_case = -tf.math.log(self.pmw + ((1.0-self.pmw)*zero_nb)+epsilon)
            result = tf.where(tf.less(y_true, 1e-8), zero_case, nb_case)
            r = self.lambda_*tf.square(self.pmw)
            result += r

 
            result = tf.reduce_mean(result)

            result = _nan_to_infinite(result)

        return result

In [ ]:
# Create the basic Autoencoder
def create_autoencoder(loss_type="mse"):
    # Step 1, defining the autoencoder
    input = Input(shape=(network_keywords["input_size"],), name='counts')
    sf = Input(shape=(1,), name='sf')

    x = Dense(64, activation=None, kernel_initializer=network_keywords["init"], name="encoder0")(input)

    x = BatchNormalization(center=True, scale=False)(x)

    x = Activation(network_keywords["activation"], name="relu_encoder0")(x)

    x = Dense(32, activation=None, kernel_initializer=network_keywords["init"], name="latent0")(x)

    x = BatchNormalization(center=True, scale=False)(x)

    x = Activation(network_keywords["activation"], name="relu_latent0")(x)

    x = Dense(64, activation=None, kernel_initializer=network_keywords["init"], name="decoder0")(x)

    x = BatchNormalization(center=True, scale=False)(x)

    x = Activation(network_keywords["activation"], name="relu_decoder0")(x)

    if loss_type == "mse":
        mean = Dense(network_keywords["input_size"], kernel_initializer=network_keywords["init"],
                        name='mean')(x)

        output = Multiply([mean, sf])

        return keras.Model(inputs=[input,sf], outputs=output, name="mse_net"), None

    elif loss_type == "nb":
        var = Dense(network_keywords["input_size"], activation=VarActivation,
                           kernel_initializer=network_keywords["init"],
                           name='variance')(x)

        mean = Dense(network_keywords["input_size"], activation=MeanActivation, 
                     kernel_initializer=network_keywords["init"],
                       name='mean')(x)
        
        output = Multiply([mean, sf])
        output = SliceLayer(0, name='slice')([output, var])

        nb = NbLoss(theta=var)
        loss = nb.loss

        return keras.Model(inputs=[input, sf], outputs=output, name="nb_net"), loss
    
    elif loss_type == "zinb":
        # pmw = point mass weight
        pmw = Dense(network_keywords["input_size"], activation='sigmoid', kernel_initializer=network_keywords["init"],
                       name='pi')(x)
        
        var = Dense(network_keywords["input_size"], activation=VarActivation,
                           kernel_initializer=network_keywords["init"],
                           name='variance')(x)

        mean = Dense(network_keywords["input_size"], activation=MeanActivation, kernel_initializer=network_keywords["init"],
                       name='mean')(x)
        
        output = Multiply([mean, sf])
        output = SliceLayer(0, name='slice')([output, var, pmw])

        zinb = ZiNbLoss(pmw, theta=var, lambda_=0)
        loss = zinb.loss

        return keras.Model(inputs=[input, sf], outputs=output, name="zinb_net"), loss

In [ ]:
def run(loss_type="zinb"): # loss_type can be mse, nb, zinb
    autoencoder, loss_fn = create_autoencoder(loss_type=loss_type)
    
    if not loss_fn and loss_type == "mse":
        loss_fn = MeanSquaredError()

    autoencoder.compile(
        optimizer=keras.optimizers.RMSprop(lr=1e-3, clipvalue=5),
        loss=loss_fn,) #loss_fn
    
    input_data = {'counts': adata.X, 'sf': adata.obs.sf}
    output_data = adata.raw.X
    
    verbose=False
    lr_patience=10
    es_patience=15
    
    callbacks = [ReduceLROnPlateau(monitor='val_loss', patience=lr_patience, verbose=verbose), EarlyStopping(monitor='val_loss', patience=es_patience, verbose=verbose)]
    autoencoder.fit(input_data, output_data, batch_size=32, epochs=300, validation_split=0.1, callbacks=callbacks)
    
    result = autoencoder.predict({'counts': adata.X, 'sf': adata.obs.sf})
    
    denoised_adata = sc.AnnData(result,obs=pd.DataFrame(y, columns=[obs_label_column]))
    
    denoised_adata_norm = denoised_adata.copy()
    
    sc.pp.normalize_total(denoised_adata_norm)
    sc.pp.log1p(denoised_adata_norm)
    
    if loss_type == "mse":
        denoised_adata_norm.X = np.nan_to_num(denoised_adata_norm.X, nan=0.1, posinf=0, neginf=0)
        
    return autoencoder, denoised_adata_norm

In [ ]:
autoencoder, denoised_adata_norm = run(loss_type="zinb")

sc.pp.pca(denoised_adata_norm)
    
sc.pl.pca_scatter(denoised_adata_norm, color='Group', size=20, title="Denoised(ZINB)", show=False, legend_loc='none')

In [ ]:
from sklearn.metrics import silhouette_score
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:

# Generate silhouette scores barplot
d = {"No Dropout": [], "Dropout": [],"ZINB": [], "NB": [], "MSE": [], }
# sim_raw_norm = sim_raw.copy()

d["No Dropout"].append(silhouette_score(sim_true_norm.obsm['X_pca'][:, :2], 
                 sim_true_norm.obs.Group))

d["Dropout"].append(silhouette_score(sim_raw_norm.obsm['X_pca'][:, :2], 
                 sim_raw_norm.obs.Group))

for i in range(0, 1):    
    ae, zinb_r = run(loss_type="zinb")
    
    sc.pp.pca(zinb_r)
    
    d["ZINB"].append(silhouette_score(zinb_r.obsm['X_pca'][:, :2], 
                 zinb_r.obs.Group))
    
    ae, nb_r = run(loss_type="nb")
        
    sc.pp.pca(nb_r)
    
    d["NB"].append(silhouette_score(nb_r.obsm['X_pca'][:, :2], 
                 nb_r.obs.Group))
    
    ae, mse_r = run(loss_type="mse")
    
    sc.pp.pca(mse_r)
    
    d["MSE"].append(silhouette_score(mse_r.obsm['X_pca'][:, :2], 
                 nb_r.obs.Group))

In [ ]:
df = pd.DataFrame([(k, v) for k, values in d.items() for v in values], columns=['Category', 'Silhouette Score'])

# Display the DataFrame
df

In [ ]:
df
plt.xticks(rotation=30)
sns.barplot(data=df, x="Category", y="Silhouette Score")